# Part 1 - Extracting and Saving Data from Yelp API

## Obective

- For this CodeAlong, we will be working with the Yelp API. 
- You will use the the Yelp API to search your home town for a cuisine type of your choice.
- Next class, we will then use Plotly Express to create a map with the Mapbox API to visualize the results.
    
    

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- Efficient API Calls Lesson Link: https://login.codingdojo.com/m/376/12529/88078

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
#!pip install yelpapi

## 1. Registering for Required APIs


- Yelp: https://www.yelp.com/developers/documentation/v3/get_started


> Check the official API documentation to know what arguments we can search for: https://www.yelp.com/developers/documentation/v3/business_search

### Load Credentials and Create Yelp API Object

In [3]:
# Load API Credentials
with open('/Users/jacksonmuehlbauer/.secret/yelp_api.json') as f:
    login = json.load(f)

In [10]:
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

### Define Search Terms and File Paths

In [4]:
# set our API call parameters and filename before the first call
location = 'Minneapolis, MN'
term = 'sushi'

In [6]:
## Specify fodler for saving data
FOLDER = 'Data/'

os.makedirs(FOLDER, exist_ok=True)
# Specifying JSON_FILE filename (can include a folder)
JSON_FILE = FOLDER+f"{location.split(',')[0]}-{term}.json"

In [7]:
JSON_FILE

'Data/Minneapolis-sushi.json'

### Check if Json File exists and Create it if it doesn't

In [8]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:   
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder, exist_ok = True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    with open(JSON_FILE, 'w') as f:
        json.dump([], f)
    
    
    ## save the first page of results
else:
    print(f'[i] {JSON_FILE} already exists')
        
## If it exists, inform user


### Load JSON FIle and account for previous results

In [ ]:
## Load previous results and use len of results for offset

## set offset based on previous results


### Make the first API call to get the first page of data

- We will use this first result to check:
    - how many total results there are?
    - Where is the actual data we want to save?
    - how many results do we get at a time?


In [11]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(term = term, location = location)

In [14]:
## How many results total?
results['total']

361

- Where is the actual data we want to save?

In [19]:
biz = pd.DataFrame(results['businesses'])
biz.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,IQpWgpVj6Z4BzvcdATBI_Q,momo-sushi-minneapolis-2,Momo Sushi,https://s3-media4.fl.yelpcdn.com/bphoto/TwKhmv...,False,https://www.yelp.com/biz/momo-sushi-minneapoli...,204,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 45.00734, 'longitude': -93.24697}","[pickup, delivery]",$$,"{'address1': '1839 Central Ave NE', 'address2'...",+16127899190,(612) 789-9190,4261.697447
1,mTnoCM3BrLttWb7m9P5SQQ,wakame-sushi-and-asian-bistro-minneapolis,Wakame Sushi & Asian Bistro,https://s3-media1.fl.yelpcdn.com/bphoto/_PG51j...,False,https://www.yelp.com/biz/wakame-sushi-and-asia...,819,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 44.946802, 'longitude': -93.321968}",[],$$,"{'address1': '3070 Excelsior Blvd', 'address2'...",+16128862484,(612) 886-2484,5431.153227
2,kcf7Bc1KKk-qoGJ2QIQVvw,billy-sushi-minneapolis,Billy Sushi,https://s3-media3.fl.yelpcdn.com/bphoto/ynRfeG...,False,https://www.yelp.com/biz/billy-sushi-minneapol...,214,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.0,"{'latitude': 44.984156, 'longitude': -93.26878...",[delivery],$$$,"{'address1': '116 N First Ave', 'address2': No...",,,1627.114072
3,RFLSUYCsAAJAneScJl51gA,kura-revolving-sushi-bar-bloomington-3,Kura Revolving Sushi Bar,https://s3-media4.fl.yelpcdn.com/bphoto/cjVtPO...,False,https://www.yelp.com/biz/kura-revolving-sushi-...,39,"[{'alias': 'conveyorsushi', 'title': 'Conveyor...",4.0,"{'latitude': 44.851055585227364, 'longitude': ...",[],NaN,"{'address1': '378 N Garden', 'address2': '', '...",,,13388.384050
4,FR9ZFGmwrrxCrolxZDy6NQ,sushi-train-minneapolis,Sushi Train,https://s3-media3.fl.yelpcdn.com/bphoto/wlQO7g...,False,https://www.yelp.com/biz/sushi-train-minneapol...,345,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",3.5,"{'latitude': 44.971904, 'longitude': -93.276914}","[pickup, delivery]",$$,"{'address1': '1200 Nicollet Mall', 'address2':...",+16122598488,(612) 259-8488,1218.244125


In [20]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

- Calculate how many pages of results needed to cover the total_results

In [21]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total'])/results_per_page)
n_pages

19

In [22]:
for i in tqdm_notebook( range(1,n_pages+1)):
    ## The block of code we want to TRY to run
    try:
        time.sleep(.2)
        
        ## Read in results in progress file and check the length
        with open(JSON_FILE, 'r') as f:
            previous_results = json.load(f)
        
        ## save number of results for to use as offset
        n_results = len(previous_results)
        
        
        ## use n_results as the OFFSET 
        results = yelp_api.search_query(location = location, term = term, offset = n_results+1)

        ## append new results and save to file
        # append would only store one record, extend will store all records
        previous_results.extend(results['businesses'])
        
        with open(JSON_FILE, 'w') as f:
            json.dump(previous_results, f)

            
    ## What to do if we get an error/exception.
    except Exception as e:
        print('[!] ERROR', e)


  0%|          | 0/19 [00:00<?, ?it/s]

## Open the Final JSON File with Pandas

In [24]:
with open(JSON_FILE, 'r') as f:
    temp = json.load(f)


df = pd.DataFrame(temp)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             360 non-null    object 
 1   alias          360 non-null    object 
 2   name           360 non-null    object 
 3   image_url      360 non-null    object 
 4   is_closed      360 non-null    bool   
 5   url            360 non-null    object 
 6   review_count   360 non-null    int64  
 7   categories     360 non-null    object 
 8   rating         360 non-null    float64
 9   coordinates    360 non-null    object 
 10  transactions   360 non-null    object 
 11  price          298 non-null    object 
 12  location       360 non-null    object 
 13  phone          360 non-null    object 
 14  display_phone  360 non-null    object 
 15  distance       360 non-null    float64
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 42.7+ KB


In [25]:
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,mTnoCM3BrLttWb7m9P5SQQ,wakame-sushi-and-asian-bistro-minneapolis,Wakame Sushi & Asian Bistro,https://s3-media1.fl.yelpcdn.com/bphoto/_PG51j...,False,https://www.yelp.com/biz/wakame-sushi-and-asia...,819,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 44.946802, 'longitude': -93.321968}",[],$$,"{'address1': '3070 Excelsior Blvd', 'address2'...",+16128862484,(612) 886-2484,5431.153227
1,kcf7Bc1KKk-qoGJ2QIQVvw,billy-sushi-minneapolis,Billy Sushi,https://s3-media3.fl.yelpcdn.com/bphoto/ynRfeG...,False,https://www.yelp.com/biz/billy-sushi-minneapol...,214,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.0,"{'latitude': 44.984156, 'longitude': -93.26878...",[delivery],$$$,"{'address1': '116 N First Ave', 'address2': No...",,,1627.114072
2,RFLSUYCsAAJAneScJl51gA,kura-revolving-sushi-bar-bloomington-3,Kura Revolving Sushi Bar,https://s3-media4.fl.yelpcdn.com/bphoto/cjVtPO...,False,https://www.yelp.com/biz/kura-revolving-sushi-...,39,"[{'alias': 'conveyorsushi', 'title': 'Conveyor...",4.0,"{'latitude': 44.851055585227364, 'longitude': ...",[],NaN,"{'address1': '378 N Garden', 'address2': '', '...",,,13388.384050
3,FR9ZFGmwrrxCrolxZDy6NQ,sushi-train-minneapolis,Sushi Train,https://s3-media3.fl.yelpcdn.com/bphoto/wlQO7g...,False,https://www.yelp.com/biz/sushi-train-minneapol...,345,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",3.5,"{'latitude': 44.971904, 'longitude': -93.276914}","[delivery, pickup]",$$,"{'address1': '1200 Nicollet Mall', 'address2':...",+16122598488,(612) 259-8488,1218.244125
4,ddpjLv0P6iu7p1dRGCPWWw,sushi-takatsu-minneapolis,Sushi Takatsu,https://s3-media1.fl.yelpcdn.com/bphoto/WVD-r_...,False,https://www.yelp.com/biz/sushi-takatsu-minneap...,142,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 44.9760103, 'longitude': -93.2709...","[delivery, pickup]",$,"{'address1': '733 Marquette Ave', 'address2': ...",+16123395981,(612) 339-5981,946.209834


In [27]:
# or
df_1 = pd.read_json(JSON_FILE)
display(df_1.head())
display(df_1.info())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,mTnoCM3BrLttWb7m9P5SQQ,wakame-sushi-and-asian-bistro-minneapolis,Wakame Sushi & Asian Bistro,https://s3-media1.fl.yelpcdn.com/bphoto/_PG51j...,False,https://www.yelp.com/biz/wakame-sushi-and-asia...,819,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 44.946802, 'longitude': -93.321968}",[],$$,"{'address1': '3070 Excelsior Blvd', 'address2'...",+16128862484,(612) 886-2484,5431.153227
1,kcf7Bc1KKk-qoGJ2QIQVvw,billy-sushi-minneapolis,Billy Sushi,https://s3-media3.fl.yelpcdn.com/bphoto/ynRfeG...,False,https://www.yelp.com/biz/billy-sushi-minneapol...,214,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.0,"{'latitude': 44.984156, 'longitude': -93.26878...",[delivery],$$$,"{'address1': '116 N First Ave', 'address2': No...",,,1627.114072
2,RFLSUYCsAAJAneScJl51gA,kura-revolving-sushi-bar-bloomington-3,Kura Revolving Sushi Bar,https://s3-media4.fl.yelpcdn.com/bphoto/cjVtPO...,False,https://www.yelp.com/biz/kura-revolving-sushi-...,39,"[{'alias': 'conveyorsushi', 'title': 'Conveyor...",4.0,"{'latitude': 44.851055585227364, 'longitude': ...",[],NaN,"{'address1': '378 N Garden', 'address2': '', '...",,,13388.384050
3,FR9ZFGmwrrxCrolxZDy6NQ,sushi-train-minneapolis,Sushi Train,https://s3-media3.fl.yelpcdn.com/bphoto/wlQO7g...,False,https://www.yelp.com/biz/sushi-train-minneapol...,345,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",3.5,"{'latitude': 44.971904, 'longitude': -93.276914}","[delivery, pickup]",$$,"{'address1': '1200 Nicollet Mall', 'address2':...",+16122598488,(612) 259-8488,1218.244125
4,ddpjLv0P6iu7p1dRGCPWWw,sushi-takatsu-minneapolis,Sushi Takatsu,https://s3-media1.fl.yelpcdn.com/bphoto/WVD-r_...,False,https://www.yelp.com/biz/sushi-takatsu-minneap...,142,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 44.9760103, 'longitude': -93.2709...","[delivery, pickup]",$,"{'address1': '733 Marquette Ave', 'address2': ...",+16123395981,(612) 339-5981,946.209834


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             360 non-null    object 
 1   alias          360 non-null    object 
 2   name           360 non-null    object 
 3   image_url      360 non-null    object 
 4   is_closed      360 non-null    bool   
 5   url            360 non-null    object 
 6   review_count   360 non-null    int64  
 7   categories     360 non-null    object 
 8   rating         360 non-null    float64
 9   coordinates    360 non-null    object 
 10  transactions   360 non-null    object 
 11  price          298 non-null    object 
 12  location       360 non-null    object 
 13  phone          360 non-null    object 
 14  display_phone  360 non-null    object 
 15  distance       360 non-null    float64
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 42.7+ KB


None

In [28]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
csv_file

'Data/Minneapolis-sushi.csv.gz'

In [29]:
## Save it as a compressed csv (to save space)
## Save it as a compressed csv (to save space)
df.to_csv(csv_file, compression='gzip', index=False)

## Bonus: compare filesize with os module's `os.path.getsize`

In [30]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

JSON FILE: 355,129 Bytes
CSV.GZ FILE: 53,198 Bytes
the csv.gz is 6.675608105567878 times smaller!


## Next Class: Processing the Results and Mapping 